# Data feature engineering script

Authors: [Andreas Sørensen](https://www.linkedin.com/in/a-soerensen) and [Martin Röck](https://www.linkedin.com/in/martinroeck/) - Source: https://doi.org/10.5281/zenodo.5895051

In [1]:
# SETUP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
# set data frame printing to unlimited rows and columns
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore') #gets rid of the 'red' warnings

In [ ]:
print("Feature engineering data for analysis...")

# 1. Import and tidy data

**Import data from "Data_Concat" csv to pandas dataframe**

In [4]:
# IMPORT DATA FROM DATA:CONCAT CSV-FILE

filename = 'EU-ECB_dataset_concatenated.csv'
df = pd.read_csv('00_data/2_data_concatenated/'+filename, delimiter = ',')

**Rename Ökobau to Oekobau.dat**

In [5]:
#Replace entries with English translation
df.lca_database.replace('Ökobau','Oekobau.dat',inplace=True)

**Harmonize 'no data' to 'No data'**

In [6]:
df.replace('no data','No data',inplace=True)

**Exclude renovation cases from dataset**

In [7]:
df = df[df.bldg_project_status != 'Renovation']

**Clean 'bldg_year_complete' use '.' as decimal instead of ','**

In [8]:
#df.loc[pd.notna(df['bldg_year_complete']), 'bldg_year_complete'] = df['bldg_year_complete'].str.replace(',', '.').astype(float)
#df['bldg_year_complete'] = df['bldg_year_complete'].str.replace(',', '.').astype(float)

In [9]:
#Check if the entries are formated correctly
#df['bldg_year_complete'].unique()
#df['bldg_year_complete'].dtypes

**Change 'bldg_year_permit' and 'bldg_year_complete' dtype to integer**

OBS: THIS DOES NOT WORK YET - STILL CREATES FLOATS WHEN GOING FROM 0 to NAN

In [10]:
"""df['bldg_year_permit'] = df['bldg_year_permit'].fillna(0).astype(int)
df.loc[(df['bldg_year_permit'] == 0), 'bldg_year_permit'] = np.nan

df['bldg_year_complete'] = df['bldg_year_complete'].fillna(0).astype(int)
df.loc[(df['bldg_year_complete'] == 0), 'bldg_year_complete'] = np.nan"""

"df['bldg_year_permit'] = df['bldg_year_permit'].fillna(0).astype(int)\ndf.loc[(df['bldg_year_permit'] == 0), 'bldg_year_permit'] = np.nan\n\ndf['bldg_year_complete'] = df['bldg_year_complete'].fillna(0).astype(int)\ndf.loc[(df['bldg_year_complete'] == 0), 'bldg_year_complete'] = np.nan"

In [11]:
#print(df['bldg_year_permit'].dtypes)
#print(df['bldg_year_complete'].dtypes)

#print(df['bldg_year_permit'].unique())
#print(df['bldg_year_complete'].unique())

In [12]:
#print(df['bldg_year_permit'].dtypes)
#print(df['bldg_year_complete'].unique())

**Adjust 0 to NaN in all columns (except 'bldg_floors_bg') before feature engineering**

In [13]:
#'bldg_floors_bg' is allowed to have 0, the rest are changed to NaN

#Create list of columns without 'bldg_floors_bg'
col_list = list(df.drop('bldg_floors_bg',axis=1).columns)

#Loop over list of columns and replace 0 with NaN
for col in col_list:
    df[col].replace(to_replace=[0,"0"], value=np.nan, inplace=True)

# 2. Add admin features

**2.1 Add features (data columns) based on existing data**

In [14]:
#Add site_country_code for each country
df.loc[(df['site_country'] == 'Denmark'), 'site_country_code'] = 'DK'
df.loc[(df['site_country'] == 'France'), 'site_country_code'] = 'FR'
df.loc[(df['site_country'] == 'Austria'), 'site_country_code'] = 'AT'
df.loc[(df['site_country'] == 'Germany'), 'site_country_code'] = 'DE'
df.loc[(df['site_country'] == 'Switzerland'), 'site_country_code'] = 'CH'
df.loc[(df['site_country'] == 'United Kingdom'), 'site_country_code'] = 'UK'
df.loc[(df['site_country'] == 'Italy'), 'site_country_code'] = 'IT'
df.loc[(df['site_country'] == 'Portugal'), 'site_country_code'] = 'PT'
df.loc[(df['site_country'] == 'Ireland'), 'site_country_code'] = 'IE'
df.loc[(df['site_country'] == 'Greece'), 'site_country_code'] = 'GR'
df.loc[(df['site_country'] == 'Norway'), 'site_country_code'] = 'NR'
df.loc[(df['site_country'] == 'Belgium'), 'site_country_code'] = 'BE'
df.loc[(df['site_country'] == 'Lithuania'), 'site_country_code'] = 'LT'
df.loc[(df['site_country'] == 'Spain'), 'site_country_code'] = 'ES'
df.loc[(df['site_country'] == 'Finland'), 'site_country_code'] = 'FI'
df.loc[(df['site_country'] == 'Netherlands'), 'site_country_code'] = 'NL'
df.loc[(df['site_country'] == 'Poland'), 'site_country_code'] = 'PL'
df.loc[(df['site_country'] == 'Sweden'), 'site_country_code'] = 'SE'
df.loc[(df['site_country'] == 'Lithuania'), 'site_country_code'] = 'LT'

#Add site_country_code for Europe (CLF data)
df.loc[(df['site_country'] == 'Europe'), 'site_country_code'] = 'EU'

In [15]:
#Add admin_data_partner
df['admin_data_partner'] = df['admin_project_contact']

#Adjust PORR data for admin_project_contact
df.loc[(df['admin_project_contact'] == 'Neururer' ), 'admin_data_partner'] = 'PORR'
df.loc[(df['admin_project_contact'] == 'Wald'), 'admin_data_partner'] = 'PORR'
df.loc[(df['admin_project_contact'] == 'Auinger' ), 'admin_data_partner'] = 'PORR'
df.loc[(df['admin_project_contact'] == 'Holdefleiss'), 'admin_data_partner'] = 'PORR'
df.loc[(df['admin_project_contact'] == 'Ruhe'), 'admin_data_partner'] = 'PORR'

# 3. GHG feature fill [kgCO2] => [kgCO2/m2/year]

Add missing GHG [kgCO2/m2/year] features based on GHG [kgCO2], bldg_area_gfa and lca_RSP

**3.1 Individual modules [kgCO2] => individual modules [kgCO2e/m2/a]**

In [16]:
#Add/fill feature:
"""
"Results - Individual modules [kgCO2e/m2/a]"
"""
#based on feature(s):
"""
"Results - Individual modules [kgCO2e]", "bldg_area_gfa" and "lca_RSP"
"""
#This is done by checking that the wanted feature is NaN to avoid overwriting existing data,
#if is NaN, it is checked whether the necessary values to fill it are not NaN,
#if these are not NaN, they are used to calculate results in [kgCO2e/m2/a].
#If wanted feature is already present or necessary values are NaN, the row is skipped.

#A1
df.loc[pd.isnull(df['GHG_A1_m2a']) & pd.notna(df['GHG_A1']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']),'GHG_A1_m2a'] = df['GHG_A1']/df['bldg_area_gfa']/df['lca_RSP']

#A2
df.loc[pd.isnull(df['GHG_A2_m2a']) & pd.notna(df['GHG_A2']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A2_m2a'] = df['GHG_A2']/df['bldg_area_gfa']/df['lca_RSP']

#A3
df.loc[pd.isnull(df['GHG_A3_m2a']) & pd.notna(df['GHG_A3']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A3_m2a'] = df['GHG_A3']/df['bldg_area_gfa']/df['lca_RSP']

#A4
df.loc[pd.isnull(df['GHG_A4_m2a']) & pd.notna(df['GHG_A4']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A4_m2a'] = df['GHG_A4']/df['bldg_area_gfa']/df['lca_RSP']

#A5
df.loc[pd.isnull(df['GHG_A5_m2a']) & pd.notna(df['GHG_A5']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A5_m2a'] = df['GHG_A5']/df['bldg_area_gfa']/df['lca_RSP']

#B1
df.loc[pd.isnull(df['GHG_B1_m2a']) & pd.notna(df['GHG_B1']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B1_m2a'] = df['GHG_B1']/df['bldg_area_gfa']/df['lca_RSP']

#B2
df.loc[pd.isnull(df['GHG_B2_m2a']) & pd.notna(df['GHG_B2']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B2_m2a'] = df['GHG_B2']/df['bldg_area_gfa']/df['lca_RSP']

#B3
df.loc[pd.isnull(df['GHG_B3_m2a']) & pd.notna(df['GHG_B3']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B3_m2a'] = df['GHG_B3']/df['bldg_area_gfa']/df['lca_RSP']

#B4
df.loc[pd.isnull(df['GHG_B4_m2a']) & pd.notna(df['GHG_B4']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B4_m2a'] = df['GHG_B4']/df['bldg_area_gfa']/df['lca_RSP']

#B5
df.loc[pd.isnull(df['GHG_B5_m2a']) & pd.notna(df['GHG_B5']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B5_m2a'] = df['GHG_B5']/df['bldg_area_gfa']/df['lca_RSP']

#B6
df.loc[pd.isnull(df['GHG_B6_m2a']) & pd.notna(df['GHG_B6']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B6_m2a'] = df['GHG_B6']/df['bldg_area_gfa']/df['lca_RSP']

#B7
df.loc[pd.isnull(df['GHG_B7_m2a']) & pd.notna(df['GHG_B7']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B7_m2a'] = df['GHG_B7']/df['bldg_area_gfa']/df['lca_RSP']

#C1
df.loc[pd.isnull(df['GHG_C1_m2a']) & pd.notna(df['GHG_C1']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C1_m2a'] = df['GHG_C1']/df['bldg_area_gfa']/df['lca_RSP']

#C2
df.loc[pd.isnull(df['GHG_C2_m2a']) & pd.notna(df['GHG_C2']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C2_m2a'] = df['GHG_C2']/df['bldg_area_gfa']/df['lca_RSP']

#C3
df.loc[pd.isnull(df['GHG_C3_m2a']) & pd.notna(df['GHG_C3']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C3_m2a'] = df['GHG_C3']/df['bldg_area_gfa']/df['lca_RSP']

#C4
df.loc[pd.isnull(df['GHG_C4_m2a']) & pd.notna(df['GHG_C4']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C4_m2a'] = df['GHG_C4']/df['bldg_area_gfa']/df['lca_RSP']

#D
df.loc[pd.isnull(df['GHG_D_m2a']) & pd.notna(df['GHG_D']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_D_m2a'] = df['GHG_D']/df['bldg_area_gfa']/df['lca_RSP']

**3.2 Aggregated modules [kgCO2] => aggregated modules [kgCO2e/m2/a]**

In [17]:
#Add/fill feature:
"""
"Results - Aggregated modules [kgCO2e/m2/a]"
"""
#based on feature(s):
"""
"Results - Aggregated modules [kgCO2e]", "bldg_area_gfa" and "lca_RSP"
"""
#This is done by checking that the wanted feature is NaN to avoid overwriting existing data,
#if is NaN, it is checked whether the necessary values to fill it are not NaN,
#if these are not NaN, they are used to calculate results in [kgCO2e/m2/a].
#If wanted feature is already present or necessary values are NaN, the row is skipped.

#A1-A3
df.loc[pd.isnull(df['GHG_A123_m2a']) & pd.notna(df['GHG_A123']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']),'GHG_A123_m2a'] = df['GHG_A123']/df['bldg_area_gfa']/df['lca_RSP']

#A4-A5
df.loc[pd.isnull(df['GHG_A45_m2a']) & pd.notna(df['GHG_A45']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A45_m2a'] = df['GHG_A45']/df['bldg_area_gfa']/df['lca_RSP']

#A1-A5
df.loc[pd.isnull(df['GHG_A12345_m2a']) & pd.notna(df['GHG_A12345']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_A12345_m2a'] = df['GHG_A12345']/df['bldg_area_gfa']/df['lca_RSP']

#B1-B4
df.loc[pd.isnull(df['GHG_B1234_m2a']) & pd.notna(df['GHG_B1234']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B1234_m2a'] = df['GHG_B1234']/df['bldg_area_gfa']/df['lca_RSP']

#B1-B5
df.loc[pd.isnull(df['GHG_B12345_m2a']) & pd.notna(df['GHG_B12345']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B12345_m2a'] = df['GHG_B12345']/df['bldg_area_gfa']/df['lca_RSP']

#B6-B7
df.loc[pd.isnull(df['GHG_B67_m2a']) & pd.notna(df['GHG_B67']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B67_m2a'] = df['GHG_B67']/df['bldg_area_gfa']/df['lca_RSP']

#B1-B7
df.loc[pd.isnull(df['GHG_B1234567_m2a']) & pd.notna(df['GHG_B1234567']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_B1234567_m2a'] = df['GHG_B1234567']/df['bldg_area_gfa']/df['lca_RSP']

#C1-C2
df.loc[pd.isnull(df['GHG_C12_m2a']) & pd.notna(df['GHG_C12']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C12_m2a'] = df['GHG_C12']/df['bldg_area_gfa']/df['lca_RSP']

#C3-C4
df.loc[pd.isnull(df['GHG_C34_m2a']) & pd.notna(df['GHG_C34']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C34_m2a'] = df['GHG_C34']/df['bldg_area_gfa']/df['lca_RSP']

#C1-C4
df.loc[pd.isnull(df['GHG_C1234_m2a']) & pd.notna(df['GHG_C1234']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C1234_m2a'] = df['GHG_C1234']/df['bldg_area_gfa']/df['lca_RSP']

#C3-C4 + D
df.loc[pd.isnull(df['GHG_C34_D_m2a']) & pd.notna(df['GHG_C34_D']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C34_D_m2a'] = df['GHG_C34_D']/df['bldg_area_gfa']/df['lca_RSP']

#C1-C4 + D
df.loc[pd.isnull(df['GHG_C1234_D_m2a']) & pd.notna(df['GHG_C1234_D']) & pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_C1234_D_m2a'] = df['GHG_C1234_D']/df['bldg_area_gfa']/df['lca_RSP']

**3.3 Totals [kgCO2] => totals [kgCO2e/m2/a]**

In [18]:
#Add/fill feature:
"""
"Results - Totals [kgCO2e/m2/a]"
"""
#based on feature(s):
"""
"Results - Totals [kgCO2e]", "bldg_area_gfa" and "lca_RSP"
"""
#This is done by checking that the wanted feature is NaN to avoid overwriting existing data,
#if is NaN, it is checked whether the necessary values to fill it are not NaN,
#if these are not NaN, they are used to calculate results in [kgCO2e/m2/a].
#If wanted feature is already present or necessary values are NaN, the row is skipped.

#sum_embodied_m2a
df.loc[pd.isnull(df['GHG_sum_em_m2a']) & pd.notna(df['GHG_sum_em'])& pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_sum_em_m2a'] = df['GHG_sum_em']/df['bldg_area_gfa']/df['lca_RSP']

#sum_operational_m2a
df.loc[pd.isnull(df['GHG_sum_op_m2a']) & pd.notna(df['GHG_sum_op'])& pd.notna(df['bldg_area_gfa']) & pd.notna(df['lca_RSP']), 'GHG_sum_op_m2a'] = df['GHG_sum_op']/df['bldg_area_gfa']/df['lca_RSP']

# 4. Add aggregated life cycle stages

Both in units of [kgCO2/m2/a] and [kgCO2/capita/a]

In [19]:
#Create aggregated life cycle stage features
df['GHG_A_m2a'] = np.nan
df['GHG_B_em_m2a'] = np.nan
df['GHG_B_op_m2a'] = np.nan
df['GHG_C_m2a'] = np.nan

df['GHG_A_capita_a'] = np.nan
df['GHG_B_em_capita_a'] = np.nan
df['GHG_B_op_capita_a'] = np.nan
df['GHG_C_capita_a'] = np.nan

In [20]:
#Add/fill GHG [kgCO2/m2/a]

#GHG_A:

"""
#GHG_A1
#GHG_A2
#GHG_A3 } GHG_A123
#GHG_A4
#GHG_A5 } GHG_A45  } GHG_A
"""

#If GHG_A123_m2a is NaN, use GHG_A1_m2a + GHG_A2_m2a + GHG_A3_m2a
df.loc[pd.isnull(df['GHG_A123_m2a']), 'GHG_A123_m2a'] = df['GHG_A1_m2a'].fillna(0)+df['GHG_A2_m2a'].fillna(0)+df['GHG_A3_m2a'].fillna(0)

#If GHG_A45_m2a is NaN, use GHG_A4_m2a + GHG_A5_m2a
df.loc[pd.isnull(df['GHG_A45_m2a']), 'GHG_A45_m2a'] = df['GHG_A4_m2a'].fillna(0)+df['GHG_A5_m2a'].fillna(0)

#If GHG_A_m2a is NaN, use GHG_A123_m2a + GHG_A45_m2a
df.loc[pd.isnull(df['GHG_A_m2a']), 'GHG_A_m2a'] = df['GHG_A123_m2a'].fillna(0)+df['GHG_A45_m2a'].fillna(0)

##########################################

#GHG_B_em:

"""
#GHG_B1
#GHG_B2
#GHG_B3
#GHG_B4 } GHG_B1234  
#GHG_B5 } GHG_B5    } GHG_B_em
"""

#If GHG_B1234_m2a is NaN, use GHG_B1_m2a + GHG_B2_m2a + GHG_B3_m2a + GHG_B4_m2a
df.loc[pd.isnull(df['GHG_B1234_m2a']), 'GHG_B1234_m2a'] = df['GHG_B1_m2a'].fillna(0)+df['GHG_B2_m2a'].fillna(0)+df['GHG_B3_m2a'].fillna(0)+df['GHG_B4_m2a'].fillna(0)

#If GHG_B_em_m2a is NaN, use GHG_B1234_m2a + GHG_B5_m2a
df.loc[pd.isnull(df['GHG_B_em_m2a']), 'GHG_B_em_m2a'] = df['GHG_B1234_m2a'].fillna(0)+df['GHG_B5_m2a'].fillna(0)

##########################################

#GHG_B_op:

#GHG_B6 } GHG_B_op

#If GHG_B_op_m2a is NaN, use GHG_B6_m2a
df.loc[pd.isnull(df['GHG_B_op_m2a']), 'GHG_B_op_m2a'] = df['GHG_B6_m2a'].fillna(0)+df['GHG_B7_m2a'].fillna(0)

#If GHG_B67_m2a is NaN, use GHG_B6_m2a + GHG_B7_m2a
df.loc[pd.isnull(df['GHG_B67_m2a']), 'GHG_B67_m2a'] = df['GHG_B6_m2a'].fillna(0)+df['GHG_B7_m2a'].fillna(0)

#If GHG_B67_m2a is still NaN, use GHG_sum_op_m2a
df.loc[(df['GHG_B67_m2a'] == 0), 'GHG_B67_m2a'] = df['GHG_sum_op_m2a']

##########################################

#GHG_C:

"""
#GHG_C1
#GHG_C2 } GHG_C12
#GHG_C3
#GHG_C4 } GHG_C34 } GHG_C1234 } GHG_C
"""

#If GHG_C12_m2a is NaN, use GHG_C1_m2a + GHG_C2_m2a
df.loc[pd.isnull(df['GHG_C12_m2a']), 'GHG_C12_m2a'] = df['GHG_C1_m2a'].fillna(0)+df['GHG_C2_m2a'].fillna(0)

#If GHG_C34_m2a is NaN, use GHG_C3_m2a + GHG_C4_m2a
df.loc[pd.isnull(df['GHG_C34_m2a']), 'GHG_C34_m2a'] = df['GHG_C3_m2a'].fillna(0)+df['GHG_C4_m2a'].fillna(0)

#If GHG_1234_m2a is NaN, use GHG_C12_m2a + GHG_C34_m2a
df.loc[pd.isnull(df['GHG_C1234_m2a']), 'GHG_C1234_m2a'] = df['GHG_C12_m2a'].fillna(0)+df['GHG_C34_m2a'].fillna(0)

#If GHG_C_m2a is NaN, use GHG_C1234_m2a
df.loc[pd.isnull(df['GHG_C_m2a']), 'GHG_C_m2a'] = df['GHG_C1234_m2a'].fillna(0)

##########################################

#GHG_sum_em_m2a:

"""
#GHG_A
#GHG_B_em } GHG_sum_em
#GHG_C
"""

#If GHG_sum_em_m2a is NaN, use GHG_A_m2a + GHG_B_em_m2a + GHG_C_m2a
df.loc[pd.isnull(df['GHG_sum_em_m2a']), 'GHG_sum_em_m2a'] = df['GHG_A_m2a'].fillna(0)+df['GHG_B_em_m2a'].fillna(0)+df['GHG_C_m2a'].fillna(0)

##########################################

#Reinstate NaN instead of 0

#Create list of columns to replace
col_list = ['GHG_A1_m2a',
            'GHG_A2_m2a',
            'GHG_A3_m2a',
            'GHG_A4_m2a',
            'GHG_A5_m2a',
            'GHG_A123_m2a',
            'GHG_A45_m2a',
            
            'GHG_B1_m2a',
            'GHG_B2_m2a',
            'GHG_B3_m2a',
            'GHG_B4_m2a',
            'GHG_B5_m2a',
            'GHG_B6_m2a',
            'GHG_B7_m2a',
            'GHG_B1234_m2a',
            
            'GHG_C1_m2a',
            'GHG_C2_m2a',
            'GHG_C3_m2a',
            'GHG_C4_m2a',
            'GHG_C12_m2a',
            'GHG_C34_m2a'
           ]

#Loop over list of columns and replace 0 with NaN
for col in col_list:
    df[col].replace(to_replace=[0,"0"], value=np.nan, inplace=True)

In [21]:
#Add/fill GHG [kgCO2/capita/a]

#GHG_A_capita_a:
df['GHG_A_capita_a'] = df['GHG_A_m2a']*df['bldg_area_gfa']/df['bldg_users_total']

#GHG_B_em_capita_a:
df['GHG_B_em_capita_a'] = df['GHG_B_em_m2a']*df['bldg_area_gfa']/df['bldg_users_total']

#GHG_B_op_capita_a:
df['GHG_B_op_capita_a'] = df['GHG_B_op_m2a']*df['bldg_area_gfa']/df['bldg_users_total']

#GHG_C_capita_a:
df['GHG_C_capita_a'] = df['GHG_C_m2a']*df['bldg_area_gfa']/df['bldg_users_total']

#GHG_sum_em_capita_a:
df['GHG_sum_em_capita_a'] = df['GHG_sum_em_m2a']*df['bldg_area_gfa']/df['bldg_users_total']

# 5. Add new aggregated scope features

**5.1 Life cycle stage (LCS) scope feature**

5.1.1 Life cycle stages

We need to build a string based on:
<br>- **A**
<br>- **B**
<br>- **C**
<br>- **D**

In [22]:
#Initiate new feature
df['scope_LCS'] = ""

#Add 'A' to string if one of the conditions are true
df.loc[(df['scope_LCS_A123'] == 'Yes')|
       (df['scope_LCS_A4'] == 'Yes')|
       (df['scope_LCS_A5'] == 'Yes'), 'scope_LCS'] = 'A'

#Add 'B' to string if one of the conditions are true
df.loc[(df['scope_LCS_B1'] == 'Yes')|
       (df['scope_LCS_B2'] == 'Yes')|
       (df['scope_LCS_B3'] == 'Yes')|
       (df['scope_LCS_B4'] == 'Yes')|
       (df['scope_LCS_B5'] == 'Yes')|
       (df['scope_LCS_B6'] == 'Yes')|
       (df['scope_LCS_B7'] == 'Yes'), 'scope_LCS'] = df['scope_LCS']+'B'

#Add 'C' to string if one of the conditions are true
df.loc[(df['scope_LCS_C1'] == 'Yes')|
       (df['scope_LCS_C2'] == 'Yes')|
       (df['scope_LCS_C3'] == 'Yes')|
       (df['scope_LCS_C4'] == 'Yes'), 'scope_LCS'] = df['scope_LCS']+'C'

#Add 'D' to string if one of the conditions are true
df.loc[(df['scope_LCS_D'] == 'Yes'), 'scope_LCS'] = df['scope_LCS']+'D'

#If none of the conditions are true, data will be empty

5.1.2 Life cycle modules

We need to build a string based on:
<br>- A1-3: **P**roduction
<br>- A4-5: **C**onstruction process
<br>- B1-4: **M**aintenance, repair, replacement
<br>- B5: **R**efurbishment
<br>- B6-7: **O**perational energy & water use
<br>- C1-2: **D**econstruction, transport
<br>- C3-4: **W**aste processing and disposal	

In [23]:
#Initiate new feature
df['scope_LCM'] = ""

#Add 'P' to string if the condition is true
df.loc[(df['scope_LCS_A123'] == 'Yes'), 'scope_LCM'] = 'P'

#Add 'C' to string if one of the conditions are true
df.loc[(df['scope_LCS_A4'] == 'Yes')|
       (df['scope_LCS_A5'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'C'

#Add 'M' to string if one of the conditions are true
df.loc[(df['scope_LCS_B1'] == 'Yes')|
       (df['scope_LCS_B2'] == 'Yes')|
       (df['scope_LCS_B3'] == 'Yes')|
       (df['scope_LCS_B4'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'M'

#Add 'R' to string if one of the conditions are true
df.loc[(df['scope_LCS_B5'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'R'

#Add 'O' to string if one of the conditions are true
df.loc[(df['scope_LCS_B6'] == 'Yes')|
       (df['scope_LCS_B7'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'O'

#Add 'D' to string if one of the conditions are true
df.loc[(df['scope_LCS_C1'] == 'Yes')|
       (df['scope_LCS_C2'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'D'

#Add 'W' to string if one of the conditions are true
df.loc[(df['scope_LCS_C3'] == 'Yes')|
       (df['scope_LCS_C4'] == 'Yes'), 'scope_LCM'] = df['scope_LCM']+'W'

#If none of the conditions are true, data will be empty

**5.2 Building parts scope feature**

We need to build a string based on:
<br>- **G**round (1) (i.e. substructure, foundation, basement walls, etc.)
<br>- **L**oad-bearing structure (2) (i.e. structural frame, walls, floors, roofs, etc.)
<br>- **E**nvelope (3, 4) (i.e. openings, ext. finishes, etc.)
<br>- **I**nternal (4) (i.e. partitions, int. finishes, etc.)
<br>- **S**ervices (5,6) (i.e. mechanical, electrical, renew. energy, etc.)
<br>- **A**ppliances (7,8) (i.e. fixed facilities, mobile fittings, etc.)


In [24]:
#Initiate new feature
df['scope_parts'] = ""

#Add 'G' to string if the condition is true
df.loc[(df['scope_parts_1_ground'] == 'Yes'), 'scope_parts'] = 'G'

#Add 'L' to string if the condition is true
df.loc[(df['scope_parts_2_structure'] == 'Yes'), 'scope_parts'] = df['scope_parts']+'L'

#Add 'E' to string if one of the conditions are true
df.loc[(df['scope_parts_3_secondary'] == 'Yes'), 'scope_parts'] = df['scope_parts']+'E'

#Add 'I' to string if the condition is true
df.loc[(df['scope_parts_4_finishes'] == 'Yes'), 'scope_parts'] = df['scope_parts']+'I'

#Add 'S' to string if one of the conditions are true
df.loc[(df['scope_parts_5_mechanical'] == 'Yes')|
       (df['scope_parts_6_electrical'] == 'Yes')|
       (df['scope_parts_6+_renewables'] == 'Yes'), 'scope_parts'] = df['scope_parts']+'S'

#Add 'A' to string if one of the conditions are true
df.loc[(df['scope_parts_7_facilities'] == 'Yes')|
       (df['scope_parts_8_fittings'] == 'Yes'), 'scope_parts'] = df['scope_parts']+'A'

#If none of the conditions are true, data will be empty

# 6. Add material/mass features

**Add/fill feature: "inv_mat_mass_total_m2"
<br>based on feature(s): "inv_mat_mass_total" & "bldg_area_gfa"**

In [25]:
#total mass per m2
df.loc[pd.notna(df['inv_mat_mass_total']) & pd.notna(df['bldg_area_gfa']), 'inv_mat_mass_total_m2'] = df['inv_mat_mass_total']/df['bldg_area_gfa']

**Add/fill feature: "inv_mat_mass_total_capita"
<br>based on feature(s): "inv_mat_mass_total" & "bldg_users_total"**

In [26]:
#total mass per capita
df.loc[pd.notna(df['inv_mat_mass_total']) & pd.notna(df['bldg_users_total']), 'inv_mat_mass_total_capita'] = df['inv_mat_mass_total']/df['bldg_users_total']

**Create new material quantity and intensity features**

Create features based on the following categories
<br>
<br>**Custom mass categories:**
<br>Aluminium (e.g., alum alloy)
<br>Bamboo
<br>Brass, copper
<br>Cement mortar, plaster
<br>Ceramics (e.g., fired clay bricks)
<br>Concrete reinforced
<br>Concrete w/o reinforcement
<br>Earth (e.g., unfired clay, adobe, rammed earth, etc.)
<br>EPS or XPS (insulation)
<br>Fungi (e.g. Mycelium)
<br>Glass (single, double, triple)
<br>Metals (iron, steel)
<br>Plastics (various: PC, PE, PP, PU, PVC)
<br>Steel (reinforcement)
<br>Stone (granite, limestone, etc)
<br>Stone wool (e.g. insulation)
<br>Straw or hemp (e.g., strawbale)
<br>Timber, wood
<br>Other
<br>No data
<br>
<br>**EUROSTAT mass categories:**
<br>1. Metal materials
<br>2. Non-metallic minerals
<br>3. Fossil energy materials
<br>4. Biomass based materials
<br>No data

Custom categories are filled in using the data in the dataset.
<br>The EUROSTAT categories are infered by looking at the custom categories and using the associations from the DataCollection 'meta' sheet.

**Custom mass categories**

In [27]:
#Create list with existing material category labels
mat_category_list = ['Aluminium (e.g., alum alloy)',
                    'Bamboo',
                    'Brass, copper',
                    'Cement mortar, plaster',
                    'Ceramics (e.g., fired clay bricks)',
                    'Concrete reinforced',
                    'Concrete w/o reinforcement',
                    'Earth (e.g., unfired clay, adobe, rammed earth, etc.)',
                    'EPS or XPS (insulation)',
                    'Fungi (e.g. Mycelium)',
                    'Glass (single, double, triple)',
                    'Metals (iron, steel)',
                    'Plastics (various: PC, PE, PP, PU, PVC)',
                    'Steel (reinforcement)',
                    'Stone (granite, limestone, etc)',
                    'Stone wool (e.g. insulation)',
                    'Straw or hemp (e.g., strawbale)',
                    'Timber, wood',
                    'Other']

#Iterate through each columns label and create a new column with the sum of the mass of the specified material category
for mat_category in mat_category_list:

    #Initiate new column
    df[mat_category] = 0

    #Check 1st material and add if conditions are met
    df.loc[(df['inv_mat_1_type'] == mat_category) & pd.notna(df['inv_mat_1_mass']), mat_category] = df[mat_category] + df['inv_mat_1_mass']

    #Check 2nd material and add if conditions are met
    df.loc[(df['inv_mat_2_type'] == mat_category) & pd.notna(df['inv_mat_2_mass']), mat_category] = df[mat_category] + df['inv_mat_2_mass']

    #Check 3rd material and add if conditions are met
    df.loc[(df['inv_mat_3_type'] == mat_category) & pd.notna(df['inv_mat_3_mass']), mat_category] = df[mat_category] + df['inv_mat_3_mass']

    #Check 4th material and add if conditions are met
    df.loc[(df['inv_mat_4_type'] == mat_category) & pd.notna(df['inv_mat_4_mass']), mat_category] = df[mat_category] + df['inv_mat_4_mass']

    #Check 5th material and add if conditions are met
    df.loc[(df['inv_mat_5_type'] == mat_category) & pd.notna(df['inv_mat_5_mass']), mat_category] = df[mat_category] + df['inv_mat_5_mass']

#Rename the category labels into simplified versions
dict_1 = {
'Aluminium (e.g., alum alloy)':'mass_aluminium',
'Bamboo':'mass_bamboo',
'Brass, copper':'mass_brass_copper',
'Cement mortar, plaster':'mass_cement_mortar',
'Ceramics (e.g., fired clay bricks)':'mass_ceramics',
'Concrete reinforced':'mass_concrete_reinforced',
'Concrete w/o reinforcement':'mass_concrete_wo_reinforcement',
'Earth (e.g., unfired clay, adobe, rammed earth, etc.)':'mass_earth',
'EPS or XPS (insulation)':'mass_EPS_XPS',
'Fungi (e.g. Mycelium)':'mass_fungi',
'Glass (single, double, triple)':'mass_glass',
'Metals (iron, steel)':'mass_metals',
'Plastics (various: PC, PE, PP, PU, PVC)':'mass_plastics',
'Steel (reinforcement)':'mass_steel_reinforcement',
'Stone (granite, limestone, etc)':'mass_stone',
'Stone wool (e.g. insulation)':'mass_stone_wool',
'Straw or hemp (e.g., strawbale)':'mass_straw_hemp',
'Timber, wood':'mass_wood',
'Other':'mass_other'
}    

df.rename(columns = dict_1, inplace = True)

**EUROSTAT mass categories**

In [28]:
df['eurostat_metal_materials'] = (df['mass_aluminium'] + df['mass_brass_copper'] + df['mass_metals'] + df['mass_steel_reinforcement'])
df['eurostat_non-metallic_minerals'] = (df['mass_cement_mortar'] + df['mass_ceramics'] + df['mass_concrete_reinforced'] + df['mass_concrete_wo_reinforcement'] + df['mass_earth'] + df['mass_glass'] + df['mass_stone'] + df['mass_stone_wool'])
df['eurostat_fossil_energy_materials'] = (df['mass_EPS_XPS'] + df['mass_plastics'])
df['eurostat_biomass_based_materials'] = (df['mass_bamboo'] + df['mass_fungi'] + df['mass_straw_hemp'] + df['mass_wood'])

**Create sum of mass of top 5 materials**

In [29]:
df['inv_mat_mass_sum_top_5'] = 0

#Check 1st material and add if conditions are met
df.loc[pd.notna(df['inv_mat_1_mass']), 'inv_mat_mass_sum_top_5'] = df['inv_mat_mass_sum_top_5'] + df['inv_mat_1_mass']

#Check 2nd material and add if conditions are met
df.loc[pd.notna(df['inv_mat_2_mass']), 'inv_mat_mass_sum_top_5'] = df['inv_mat_mass_sum_top_5'] + df['inv_mat_2_mass']

#Check 3rd material and add if conditions are met
df.loc[pd.notna(df['inv_mat_3_mass']), 'inv_mat_mass_sum_top_5'] = df['inv_mat_mass_sum_top_5'] + df['inv_mat_3_mass']

#Check 4th material and add if conditions are met
df.loc[pd.notna(df['inv_mat_4_mass']), 'inv_mat_mass_sum_top_5'] = df['inv_mat_mass_sum_top_5'] + df['inv_mat_4_mass']

#Check 5th material and add if conditions are met
df.loc[pd.notna(df['inv_mat_5_mass']), 'inv_mat_mass_sum_top_5'] = df['inv_mat_mass_sum_top_5'] + df['inv_mat_5_mass']


**Create ratio between 'sum of mass of top 5 materials' and total mass**

In [30]:
df['inv_mat_mass_ratio'] = 100*df['inv_mat_mass_sum_top_5']/df['inv_mat_mass_total']

# 7. Add other features

**Add/fill "gfa m2 per capita" feature**

In [31]:
#gfa m2 per capita  
df.loc[pd.notna(df['bldg_area_gfa']) & pd.notna(df['bldg_users_total']), 'gfa_m2_capita'] = df['bldg_area_gfa']/df['bldg_users_total']

**Add/infer "bldg_energy_class_general" based on "inv_energy_consumption"**

In [32]:
#Existing standard
df.loc[(pd.isnull(df['bldg_energy_class_general']) | (df['bldg_energy_class_general'] == 'No data')) & pd.notna(df['inv_energy_consumption']) & (df['inv_energy_consumption'] >= 100.0), 'bldg_energy_class_general'] = 'Existing Standard'

#New standard
df.loc[(pd.isnull(df['bldg_energy_class_general']) | (df['bldg_energy_class_general'] == 'No data')) & pd.notna(df['inv_energy_consumption']) & (df['inv_energy_consumption'] > 20.0) & (df['inv_energy_consumption'] < 100.0), 'bldg_energy_class_general'] = 'New Standard'

#Advanced new standard
df.loc[(pd.isnull(df['bldg_energy_class_general']) | (df['bldg_energy_class_general'] == 'No data')) & pd.notna(df['inv_energy_consumption']) & (df['inv_energy_consumption'] <= 20.0), 'bldg_energy_class_general'] = 'New Advanced'

**Add/infer "bldg_year_complete_interval" based on "bldg_year_complete"**

In [33]:
#2020-Today
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 2020), 'bldg_year_complete_interval'] = '2020-Today'

#2015-2019
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 2015) & (df['bldg_year_complete'] <= 2019), 'bldg_year_complete_interval'] = '2015-2019'

#2010-2014
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 2010) & (df['bldg_year_complete'] <= 2014), 'bldg_year_complete_interval'] = '2010-2014'

#2005-2009
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 2005) & (df['bldg_year_complete'] <= 2009), 'bldg_year_complete_interval'] = '2005-2009'

#2000-2004
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 2000) & (df['bldg_year_complete'] <= 2004), 'bldg_year_complete_interval'] = '2000-2004'

#1995-1999
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1995) & (df['bldg_year_complete'] <= 1999), 'bldg_year_complete_interval'] = '1995-1999'

#1990-1994
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1990) & (df['bldg_year_complete'] <= 1994), 'bldg_year_complete_interval'] = '1990-1994'

#1985-1989
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1985) & (df['bldg_year_complete'] <= 1989), 'bldg_year_complete_interval'] = '1985-1989'

#1980-1984
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1980) & (df['bldg_year_complete'] <= 1984), 'bldg_year_complete_interval'] = '1980-1984'

#1975-1979
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1975) & (df['bldg_year_complete'] <= 1979), 'bldg_year_complete_interval'] = '1975-1979'

#1970-1974
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1970) & (df['bldg_year_complete'] <= 1974), 'bldg_year_complete_interval'] = '1970-1974'

#1965-1969
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1965) & (df['bldg_year_complete'] <= 1969), 'bldg_year_complete_interval'] = '1965-1969'

#1960-1964
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1960) & (df['bldg_year_complete'] <= 1964), 'bldg_year_complete_interval'] = '1960-1964'

#1955-1959
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1955) & (df['bldg_year_complete'] <= 1959), 'bldg_year_complete_interval'] = '1955-1959'

#1950-1954
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1950) & (df['bldg_year_complete'] <= 1954), 'bldg_year_complete_interval'] = '1950-1954'

#1945-1949
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1945) & (df['bldg_year_complete'] <= 1949), 'bldg_year_complete_interval'] = '1945-1949'

#1940-1944
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1940) & (df['bldg_year_complete'] <= 1944), 'bldg_year_complete_interval'] = '1940-1944'

#1935-1939
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1935) & (df['bldg_year_complete'] <= 1939), 'bldg_year_complete_interval'] = '1935-1939'

#1930-1934
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] >= 1930) & (df['bldg_year_complete'] <= 1934), 'bldg_year_complete_interval'] = '1930-1934'

#<1930
df.loc[(pd.isnull(df['bldg_year_complete_interval']) | (df['bldg_year_complete_interval'] == 'No data')) & pd.notna(df['bldg_year_complete']) & (df['bldg_year_complete'] < 1930), 'bldg_year_complete_interval'] = '<1930'

**Add aggregation of GHG results on building parts from being divided into LCS to sum of entire life cycle**

In [34]:
#P1
df.loc[pd.isnull(df['GHG_P1_sum_m2a']), 'GHG_P1_sum_m2a'] = df['GHG_P1_A123_m2a'].fillna(0) + df['GHG_P1_A45_m2a'].fillna(0) + df['GHG_P1_B1234_m2a'].fillna(0) + df['GHG_P1_B5_m2a'].fillna(0) + df['GHG_P1_C12_m2a'].fillna(0) + df['GHG_P1_C34_m2a'].fillna(0)

#P2
df.loc[pd.isnull(df['GHG_P2_sum_m2a']), 'GHG_P2_sum_m2a'] = df['GHG_P2_A123_m2a'].fillna(0) + df['GHG_P2_A45_m2a'].fillna(0) + df['GHG_P2_B1234_m2a'].fillna(0) + df['GHG_P2_B5_m2a'].fillna(0) + df['GHG_P2_C12_m2a'].fillna(0) + df['GHG_P2_C34_m2a'].fillna(0)

#P34
df.loc[pd.isnull(df['GHG_P34_sum_m2a']), 'GHG_P34_sum_m2a'] = df['GHG_P34_A123_m2a'].fillna(0) + df['GHG_P34_A45_m2a'].fillna(0) + df['GHG_P34_B1234_m2a'].fillna(0) + df['GHG_P34_B5_m2a'].fillna(0) + df['GHG_P34_C12_m2a'].fillna(0) + df['GHG_P34_C34_m2a'].fillna(0)

#P4
df.loc[pd.isnull(df['GHG_P4_sum_m2a']), 'GHG_P4_sum_m2a'] = df['GHG_P4_A123_m2a'].fillna(0) + df['GHG_P4_A45_m2a'].fillna(0) + df['GHG_P4_B1234_m2a'].fillna(0) + df['GHG_P4_B5_m2a'].fillna(0) + df['GHG_P4_C12_m2a'].fillna(0) + df['GHG_P4_C34_m2a'].fillna(0)

#P56
df.loc[pd.isnull(df['GHG_P56_sum_m2a']), 'GHG_P56_sum_m2a'] = df['GHG_P56_A123_m2a'].fillna(0) + df['GHG_P56_A45_m2a'].fillna(0) + df['GHG_P56_B1234_m2a'].fillna(0) + df['GHG_P56_B5_m2a'].fillna(0) + df['GHG_P56_C12_m2a'].fillna(0) + df['GHG_P56_C34_m2a'].fillna(0)
        
#P78
df.loc[pd.isnull(df['GHG_P78_sum_m2a']), 'GHG_P78_sum_m2a'] = df['GHG_P78_A123_m2a'].fillna(0) + df['GHG_P78_A45_m2a'].fillna(0) + df['GHG_P78_B1234_m2a'].fillna(0) + df['GHG_P78_B5_m2a'].fillna(0) + df['GHG_P78_C12_m2a'].fillna(0) + df['GHG_P78_C34_m2a'].fillna(0)

**Create variants on GHG reference units**

In [35]:
# Create variants on GHG reference units
'''
Relevant parameters to process
'GHG_sum_em',
'GHG_sum_em_m2a',
'GHG_sum_em_capita_a',

'GHG_B67_m2a' #defacto sum_op_m2a
'GHG_B_op_capita_a'
'''
#Change values from 0 to NaN

#Create list of columns without 'bldg_floors_bg'
col_list = list(df.drop('bldg_floors_bg',axis=1).columns)

#Loop over list of columns and replace 0 with NaN
for col in col_list:
    df[col].replace(to_replace=[0,"0"], value=np.nan, inplace=True)


# m2 per t CO2 (per year / full LCS)
df['GHG_sum_em_m2pertonGHG_a'] = 1 / df['GHG_sum_em_m2a'] * 1000
df['GHG_sum_em_m2pertonGHG'] = 1 / (df['GHG_sum_em_m2a'] * df['lca_RSP']) * 1000
df['GHG_sum_op_m2pertonGHG_a'] = 1 / df['GHG_B67_m2a'] * 1000
df['GHG_sum_op_m2pertonGHG'] = 1 / (df['GHG_B67_m2a'] * df['lca_RSP']) * 1000

# users per t CO2 (per year / full LCS)
df['GHG_sum_em_cappertonGHG_a'] = 1 / df['GHG_sum_em_capita_a'] * 1000
df['GHG_sum_em_cappertonGHG'] = 1 / (df['GHG_sum_em_capita_a'] * df['lca_RSP']) * 1000
df['GHG_sum_op_cappertonGHG_a'] = 1 / df['GHG_B_op_capita_a'] * 1000
df['GHG_sum_op_cappertonGHG'] = 1 / (df['GHG_B_op_capita_a'] * df['lca_RSP']) * 1000

In [36]:
#print(df['GHG_sum_em_m2pertonGHG_a'].describe().count)
#print(df['GHG_sum_em_m2pertonGHG'].describe().count)
#print(df['GHG_sum_em_cappertonGHG_a'].describe().count)
#print(df['GHG_sum_em_cappertonGHG'].describe().count)

# 8. Harmonize GHG results

### 1. Undo original APEN data harmonization

ghg_EUECB  = ghg_APEN * 50 / RSP_case

In [37]:
#METHOD 1 (DISABLED)
#Create new column for undoing of APEN data harmonization
#df['GHG_A123_m2a_APEN_undo'] = (df['GHG_A123_m2a_APEN']*50)/df['lca_RSP']

In [38]:
#METHOD2
#Or, add undoing of APEN data harmonization to existing column
#A123
df.loc[pd.notna(df['GHG_A123_m2a_APEN']) & pd.isnull(df['GHG_A123_m2a']),'GHG_A123_m2a'] = (df['GHG_A123_m2a_APEN']*50)/df['lca_RSP']
#A45
df.loc[pd.notna(df['GHG_A45_m2a_APEN']) & pd.isnull(df['GHG_A45_m2a']),'GHG_A45_m2a'] = (df['GHG_A45_m2a_APEN']*50)/df['lca_RSP']
#B1234
df.loc[pd.notna(df['GHG_B1234_m2a_APEN']) & pd.isnull(df['GHG_B1234_m2a']),'GHG_B1234_m2a'] = (df['GHG_B1234_m2a_APEN']*50)/df['lca_RSP']
#B67
df.loc[pd.notna(df['GHG_B67_m2a_APEN']) & pd.isnull(df['GHG_B67_m2a']),'GHG_B67_m2a'] = (df['GHG_B67_m2a_APEN']*50)/df['lca_RSP']
#C12
df.loc[pd.notna(df['GHG_C12_m2a_APEN']) & pd.isnull(df['GHG_C12_m2a']),'GHG_C12_m2a'] = (df['GHG_C12_m2a_APEN']*50)/df['lca_RSP']
#C34
df.loc[pd.notna(df['GHG_C34_m2a_APEN']) & pd.isnull(df['GHG_C34_m2a']),'GHG_C34_m2a'] = (df['GHG_C34_m2a_APEN']*50)/df['lca_RSP']
#D
df.loc[pd.notna(df['GHG_D_m2a_APEN']) & pd.isnull(df['GHG_D_m2a']),'GHG_D_m2a'] = (df['GHG_D_m2a_APEN']*50)/df['lca_RSP']
#sum_em
df.loc[pd.notna(df['GHG_sum_em_m2a_APEN']) & pd.isnull(df['GHG_sum_em_m2a']),'GHG_sum_em_m2a'] = (df['GHG_sum_em_m2a_APEN']*50)/df['lca_RSP']
#sum_op
df.loc[pd.notna(df['GHG_sum_op_m2a_APEN']) & pd.isnull(df['GHG_sum_op_m2a']),'GHG_sum_op_m2a'] = (df['GHG_sum_op_m2a_APEN']*50)/df['lca_RSP']

### 2. Apply harmonization procedure to all data that is already disaggregated, and compute the harmonized sum of EC

**Harmonize GHG results per LCS**
<br><br>
**EC original RSP**
<br>GHG_A_m2 = GHG_A_m2a x case_RSP
<br>GHG_B_em_m2 = GHG_B_em_m2a x case_RSP
<br>GHG_C_m2 = GHG_C_m2a x case_RSP
<br>

**EC harmonized RSP**
<br>GHG_A_m2_harm =  GHG_A_m2
<br>GHG_B_em_m2_harm = GHG_B_em_m2 x (harm_RSP/case_RSP)
<br>GHG_C_m2_harm = GHG_C_m2
<br>
<br>**EC harmonized annualized**
<br>GHG_A_m2a_harm = GHG_A_m2_harm / RSP_harm 
<br>GHG_B_em_m2a_harm = GHG_B_em_m2_harm / RSP_harm 
<br>GHG_C_m2a_harm = GHG_C_m2_harm / RSP_harm 

In [39]:
# Define harmonized life cycle study period
lca_RSP_harm = 50

### Scale up by reference study period ###

#GHG_A123_m2
df.loc[pd.notna(df['GHG_A123_m2a']) & pd.notna(df['lca_RSP']), 'GHG_A123_m2'] = df['GHG_A123_m2a'] * df['lca_RSP']

#GHG_A45_m2
df.loc[pd.notna(df['GHG_A45_m2a']) & pd.notna(df['lca_RSP']), 'GHG_A45_m2'] = df['GHG_A45_m2a'] * df['lca_RSP']

#GHG_B1234_m2
df.loc[pd.notna(df['GHG_B1234_m2a']) & pd.notna(df['lca_RSP']), 'GHG_B1234_m2'] = (df['GHG_B1234_m2a'] * df['lca_RSP'])

#GHG_B5_m2
df.loc[pd.notna(df['GHG_B5_m2a']) & pd.notna(df['lca_RSP']), 'GHG_B5_m2'] = (df['GHG_B5_m2a'] * df['lca_RSP'])

#GHG_B67_m2
df.loc[pd.notna(df['GHG_B67_m2a']) & pd.notna(df['lca_RSP']), 'GHG_B67_m2'] = (df['GHG_B67_m2a'] * df['lca_RSP'])

#GHG_C12_m2
df.loc[pd.notna(df['GHG_C12_m2a']) & pd.notna(df['lca_RSP']), 'GHG_C12_m2'] = df['GHG_C12_m2a'] * df['lca_RSP']

#GHG_C34_m2
df.loc[pd.notna(df['GHG_C34_m2a']) & pd.notna(df['lca_RSP']), 'GHG_C34_m2'] = df['GHG_C34_m2a'] * df['lca_RSP']



### Harmonize (unique for B stage) ###

#GHG_A123_m2_harm
df['GHG_A123_m2_harm'] = df['GHG_A123_m2']

#GHG_A45_m2_harm
df['GHG_A45_m2_harm'] = df['GHG_A45_m2']

#GHG_B1234_m2_harm
df['GHG_B1234_m2_harm'] = df['GHG_B1234_m2'] * (lca_RSP_harm/df['lca_RSP'])

#GHG_B5_m2_harm
df['GHG_B5_m2_harm'] = df['GHG_B5_m2'] * (lca_RSP_harm/df['lca_RSP'])

#GHG_B67_m2_harm
df['GHG_B67_m2_harm'] = df['GHG_B67_m2'] * (lca_RSP_harm/df['lca_RSP'])

#GHG_C12_m2_harm
df['GHG_C12_m2_harm'] = df['GHG_C12_m2']

#GHG_C34_m2_harm
df['GHG_C34_m2_harm'] = df['GHG_C34_m2']



### Scale harmonized back down to m2a ###

#GHG_A123_m2a_harm
df.loc[pd.notna(df['GHG_A123_m2_harm']), 'GHG_A123_m2a_harm'] = df['GHG_A123_m2_harm'] / lca_RSP_harm

#GHG_A45_m2a_harm
df.loc[pd.notna(df['GHG_A45_m2_harm']), 'GHG_A45_m2a_harm'] = df['GHG_A45_m2_harm'] / lca_RSP_harm

#GHG_B1234_m2a_harm
df.loc[pd.notna(df['GHG_B1234_m2_harm']), 'GHG_B1234_m2a_harm'] = df['GHG_B1234_m2_harm'] / lca_RSP_harm

#GHG_B5_m2a_harm
df.loc[pd.notna(df['GHG_B5_m2_harm']), 'GHG_B5_m2a_harm'] = df['GHG_B5_m2_harm'] / lca_RSP_harm

#GHG_B67_m2a_harm
df.loc[pd.notna(df['GHG_B67_m2_harm']), 'GHG_B67_m2a_harm'] = df['GHG_B67_m2_harm'] / lca_RSP_harm

#GHG_C12_m2a_harm
df.loc[pd.notna(df['GHG_C12_m2_harm']), 'GHG_C12_m2a_harm'] = df['GHG_C12_m2_harm'] / lca_RSP_harm

#GHG_C_m2a_harm
df.loc[pd.notna(df['GHG_C34_m2_harm']), 'GHG_C34_m2a_harm'] = df['GHG_C34_m2_harm'] / lca_RSP_harm

**Harmonize GHG result sums**

In [40]:
# If EC per LCS has been harmonized, use them to calculate the harmonized sum of EC
# OBS: This code will only provide reliable sums if all three stages are provided/have data.

# Calculate GHG_sum_em_m2
df['GHG_sum_em_m2'] = df['GHG_A123_m2'].fillna(0) + df['GHG_B1234_m2'].fillna(0) + df['GHG_B5_m2'].fillna(0) + df['GHG_C12_m2'].fillna(0) + df['GHG_C34_m2'].fillna(0)

# Calculate GHG_sum_em_m2_harm
df['GHG_sum_em_m2_harm'] = df['GHG_A123_m2_harm'].fillna(0) + df['GHG_A45_m2_harm'].fillna(0) + df['GHG_B1234_m2_harm'].fillna(0) + df['GHG_B5_m2_harm'].fillna(0) + df['GHG_C12_m2_harm'].fillna(0) + df['GHG_C34_m2_harm'].fillna(0)

# Calculate GHG_sum_em_m2a_harm
df['GHG_sum_em_m2a_harm'] =  df['GHG_A123_m2a_harm'].fillna(0) + df['GHG_A45_m2a_harm'].fillna(0) + df['GHG_B1234_m2a_harm'].fillna(0) + df['GHG_B5_m2a_harm'].fillna(0) + df['GHG_C12_m2a_harm'].fillna(0) + df['GHG_C34_m2a_harm'].fillna(0)

# If one of the harmonized EC per LCS is NaN, method 1 will not run
# Instead method 2 or 3 is used

### 3. Use the harmonized sum EC and harmonized disaggregated EC to compute and apply a ratio to predict harmonized disaggregated results for cases with no disaggregated results

**Disaggregate GHG results into A123, A45, B1234, B5, B67, C12 and C34 where needed**

GHG results are disaggregated by:
<br>1. Identify which cases lack disaggregated GHG results and which don't. Identify the scope_LCM of these cases.
<br>2. Use those cases to create a ratio which can predict/disaggregate new cases based on LCS_scope.
<br>3. Use the ratio to calculate the GHG results separated into life cycle stages for the cases that need it.

**1. Identify cases, scope_LCM and scope_parts**

In [41]:
# Create new boolean feature to determine whether case has disaggregated results or not
df['Bool_1'] = np.nan
df['Bool_2'] = np.nan
df['Bool_3'] = np.nan

### LACK DISAGGREGATION ###
# Find cases which lack disaggregated results, return False in the boolean feature
df.loc[pd.isnull(df['GHG_A123_m2a_harm']) & pd.isnull(df['GHG_A45_m2a_harm']) & pd.isnull(df['GHG_B1234_m2a_harm']) & pd.isnull(df['GHG_B5_m2a_harm']) & pd.isnull(df['GHG_C12_m2a_harm']) & pd.isnull(df['GHG_C34_m2a_harm']), 'Bool_1'] = 'False'

# Create new df with cases that lack disaggregated results
df_false = df.loc[df['Bool_1'].isin(['False'])]

# Print scopes
#print('Cases without disaggregated results have these LCM scopes:')
df_false_scopes_LCM = df_false['scope_LCM'].unique()
#print(df_false_scopes_LCM)
#print()

#print('Cases without disaggregated results have these parts scopes:')
df_false_scopes_parts = df_false['scope_parts'].unique()
#print(df_false_scopes_parts)
#print()

### HAVE DISAGGREGATION scope_LCM ###
# Find cases that have disaggregated results, return True in the boolean feature
df.loc[pd.notna(df['GHG_A123_m2a_harm']) | pd.notna(df['GHG_A45_m2a_harm']) | pd.notna(df['GHG_B1234_m2a_harm']) | pd.notna(df['GHG_B5_m2a_harm']) | pd.notna(df['GHG_C12_m2a_harm']) | pd.notna(df['GHG_C34_m2a_harm']), 'Bool_2'] = 'True'

# Use it to find the scopes_LCM of the relecant cases
df_true_LCM = df.loc[df['Bool_2'].isin(['True'])]
#print('We can predict cases with these LCM scopes:')
df_true_scopes_LCM = df_true_LCM['scope_LCM'].unique()
#print(df_true_scopes_LCM)
#print()

### HAVE DISAGGREGATION scope_parts ###
# Find cases that have disaggregated results, return True in the boolean feature
df.loc[pd.notna(df['GHG_A123_m2a_harm']) | pd.notna(df['GHG_A45_m2a_harm']) | pd.notna(df['GHG_B1234_m2a_harm']) | pd.notna(df['GHG_B5_m2a_harm']) | pd.notna(df['GHG_C12_m2a_harm']) | pd.notna(df['GHG_C34_m2a_harm']), 'Bool_3'] = 'True'

# Use it to find the scopes_LCM of the relecant cases
df_true_parts = df.loc[df['Bool_2'].isin(['True'])]
#print('We can predict cases with these parts scopes:')
df_true_scopes_parts = df_true_parts['scope_parts'].unique()
#print(df_true_scopes_parts)
#print()

Cases without disaggregated results have these LCM scopes:
['PMW' 'PC' 'P' 'PCMOD' 'PCO' 'PCMODW' 'PCMRODW' 'PMO' 'PMDW' 'PCOD'
 'PMODW' 'PCMROW' 'PCOW' 'PCMO' 'PCODW' 'PM' 'PO' '' 'PCMROD' 'PMOW'
 'PCMOW' 'PCM']

Cases without disaggregated results have these parts scopes:
['GLEIS' 'GLEI' 'GL' '' 'GLIS' 'IS' 'GLSA' 'GLI' 'LI' 'LEI' 'L' 'GLS'
 'GLES' 'LE' 'GLE' 'GLEISA' 'GLESA']

We can predict cases with these LCM scopes:
['PMOW' 'PMW' 'PCMODW' 'PCMDW' 'PCMW' 'PO' 'P' 'PCMRODW' 'PCMOD' 'PCODW'
 'PCMROD' 'PCO' 'PCMO' 'PODW' 'PMODW' 'PC' '']

We can predict cases with these parts scopes:
['GLES' 'GLEIS' 'GLEI' 'GLEISA' 'GLE' 'GL' 'LEISA' 'GLIS' '' 'GLS' 'GLI'
 'LEIS']



**2. Create ratios for A123, A45, B1234, B5, B67, C12 and C34 for each type of scope_LCM and scope_parts**

In [42]:
# Filter cases from df_true using list of true LCM scopes and calculate average ratios
list_ratio_1 = []
list_ratio_2 = []

for scope_LCM in df_true_scopes_LCM:
    df_temp = df_true_LCM.loc[df['scope_LCM'].isin([scope_LCM])]

    df_temp['ratio_A123']  = (df_temp['GHG_A123_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_A45']   = (df_temp['GHG_A45_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_B1234'] = (df_temp['GHG_B1234_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_B5']    = (df_temp['GHG_B5_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_C12'] = (df_temp['GHG_C12_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_C34'] = (df_temp['GHG_C34_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    
    df_temp = df_temp[['ratio_A123','ratio_A45','ratio_B1234','ratio_B5','ratio_C12','ratio_C34']].mean()

    list_ratio_1.append([str(scope_LCM),df_temp[['ratio_A123','ratio_A45','ratio_B1234','ratio_B5','ratio_C12','ratio_C34']]])
    
for scope_parts in df_true_scopes_parts:
    df_temp = df_true_parts.loc[df['scope_parts'].isin([scope_parts])]

    df_temp['ratio_A123']  = (df_temp['GHG_A123_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_A45']   = (df_temp['GHG_A45_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_B1234'] = (df_temp['GHG_B1234_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_B5']    = (df_temp['GHG_B5_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_C12'] = (df_temp['GHG_C12_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    df_temp['ratio_C34'] = (df_temp['GHG_C34_m2a_harm']/df_temp['GHG_sum_em_m2a_harm'])
    
    df_temp = df_temp[['ratio_A123','ratio_A45','ratio_B1234','ratio_B5','ratio_C12','ratio_C34']].mean()

    list_ratio_2.append([str(scope_parts),df_temp[['ratio_A123','ratio_A45','ratio_B1234','ratio_B5','ratio_C12','ratio_C34']]])
 

**3. Apply ratios and calculate disaggregated results**

In [43]:
#Extra code to convert CSTB data with lca_RSP of 50 to harm data
df.loc[(df['admin_data_partner'] == 'CSTB') & (df['lca_RSP'] == 50),'GHG_sum_em_m2a_harm'] = df['GHG_sum_em_m2a']

In [44]:
# Apply ratios for LCM to GHG_A_m2a, GHG_B_em_m2a and GHG_C_m2a where bool is false and the scopes match

for scope_LCM in df_false_scopes_LCM:
    for i in range(0,len(list_ratio_1)):
        
        scope = list_ratio_1[i][0]
        
        #Compute A123
        ratio = list_ratio_1[i][1]['ratio_A123']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_A123_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_A123_LCM'] = ratio
            
        #Compute A45
        ratio = list_ratio_1[i][1]['ratio_A45']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_A45_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_A45_LCM'] = ratio
        
        #Compute B1234
        ratio = list_ratio_1[i][1]['ratio_B1234']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_B1234_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_B1234_LCM'] = ratio
        
        #Compute B5
        ratio = list_ratio_1[i][1]['ratio_B5']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_B5_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_B5_LCM'] = ratio
        
        #Compute C12
        ratio = list_ratio_1[i][1]['ratio_C12']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_C12_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_C12_LCM'] = ratio
        
        #Compute C34
        ratio = list_ratio_1[i][1]['ratio_C34']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'GHG_C34_m2a_harm_LCM'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_LCM'] == scope), 'ratio_C34_LCM'] = ratio
        

In [45]:
# Apply ratios for parts to GHG_A_m2a, GHG_B_em_m2a and GHG_C_m2a where bool is false and the scopes match

for scope_parts in df_false_scopes_parts:
    for i in range(0,len(list_ratio_2)):
        
        scope = list_ratio_2[i][0]
        
        #Compute A123
        ratio = list_ratio_2[i][1]['ratio_A123']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_A123_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_A123_parts'] = ratio
        
        #Compute A45
        ratio = list_ratio_2[i][1]['ratio_A45']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_A45_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_A45_parts'] = ratio
        
        #Compute B1234
        ratio = list_ratio_2[i][1]['ratio_B1234']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_B1234_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_B1234_parts'] = ratio
        
        #Compute B5
        ratio = list_ratio_2[i][1]['ratio_B5']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_B5_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_B5_parts'] = ratio
        
        #Compute C12
        ratio = list_ratio_2[i][1]['ratio_C12']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_C12_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_C12_parts'] = ratio
        
        #Compute C34
        ratio = list_ratio_2[i][1]['ratio_C34']
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'GHG_C34_m2a_harm_parts'] = df['GHG_sum_em_m2a_harm']*ratio
        df.loc[(df['Bool_1'] == 'False') & (df['scope_parts'] == scope), 'ratio_C34_parts'] = ratio
        

**4. Recreate and store ratios for data that already has disaggregated results**

These ratios are not applied and are only used for comparison and analysis

In [46]:
#A123
df.loc[pd.isnull(df['ratio_A123_LCM']),'ratio_A123_LCM'] = df['GHG_A123_m2a_harm']/df['GHG_sum_em_m2a_harm']
#A45
df.loc[pd.isnull(df['ratio_A45_LCM']),'ratio_A45_LCM'] = df['GHG_A45_m2a_harm']/df['GHG_sum_em_m2a_harm']
#B1234
df.loc[pd.isnull(df['ratio_B1234_LCM']),'ratio_B1234_LCM'] = df['GHG_B1234_m2a_harm']/df['GHG_sum_em_m2a_harm']
#B5
df.loc[pd.isnull(df['ratio_B5_LCM']),'ratio_B5_LCM'] = df['GHG_B5_m2a_harm']/df['GHG_sum_em_m2a_harm']
#C12
df.loc[pd.isnull(df['ratio_C12_LCM']),'ratio_C12_LCM'] = df['GHG_C12_m2a_harm']/df['GHG_sum_em_m2a_harm']
#C34
df.loc[pd.isnull(df['ratio_C34_LCM']),'ratio_C34_LCM'] = df['GHG_C34_m2a_harm']/df['GHG_sum_em_m2a_harm']

In [47]:
#A123
df.loc[pd.isnull(df['ratio_A123_parts']),'ratio_A123_parts'] = df['GHG_A123_m2a_harm']/df['GHG_sum_em_m2a_harm']
#A45
df.loc[pd.isnull(df['ratio_A45_parts']),'ratio_A45_parts'] = df['GHG_A45_m2a_harm']/df['GHG_sum_em_m2a_harm']
#B1234
df.loc[pd.isnull(df['ratio_B1234_parts']),'ratio_B1234_parts'] = df['GHG_B1234_m2a_harm']/df['GHG_sum_em_m2a_harm']
#B5
df.loc[pd.isnull(df['ratio_B5_parts']),'ratio_B5_parts'] = df['GHG_B5_m2a_harm']/df['GHG_sum_em_m2a_harm']
#C12
df.loc[pd.isnull(df['ratio_C12_parts']),'ratio_C12_parts'] = df['GHG_C12_m2a_harm']/df['GHG_sum_em_m2a_harm']
#C34
df.loc[pd.isnull(df['ratio_C34_parts']),'ratio_C34_parts'] = df['GHG_C34_m2a_harm']/df['GHG_sum_em_m2a_harm']

# Final checks

**Fix Residential cases in bldg_use_type**

In [48]:
#Replace wrong entries
df.bldg_use_type.replace('Residential ','Residential',inplace=True)

**Replace 0 with NaN in all columns (except 'bldg_floors_bg') after feature engineering**

In [49]:
#'bldg_floors_bg' is allowed to have 0, the rest are changed to NaN

#Create list of columns without 'bldg_floors_bg'
col_list = list(df.drop('bldg_floors_bg',axis=1).columns)

#Loop over list of columns and replace 0 with NaN
for col in col_list:
    df[col].replace(to_replace=[0,"0"], value=np.nan, inplace=True)

**Correct datatypes**

In [50]:
#List of features to correct
feature_list = [
    "bldg_floors_ag",
    "bldg_floors_ag",
    "bldg_floors_bg",
    "mass_ceramics",
    "mass_concrete_reinforced",
    "mass_glass",
    "mass_metals",
    "mass_wood",
    "eurostat_biomass_based_materials",
    "lca_RSP"
]

#Change data type of features in list to numeric (float)
df[feature_list] = df[feature_list].apply(pd.to_numeric,errors='coerce')

#Data that can't be transformed to float will be set to NaN given the "errors='coerce'" argument

In [51]:
df[feature_list].dtypes

bldg_floors_ag                      float64
bldg_floors_ag                      float64
bldg_floors_bg                      float64
mass_ceramics                       float64
mass_concrete_reinforced            float64
mass_glass                          float64
mass_metals                         float64
mass_wood                           float64
eurostat_biomass_based_materials    float64
lca_RSP                             float64
dtype: object

# Create dataframe for export

In [52]:
#Reorder columns in dataframe using double brackets (columns that aren't called are dropped):

df = df[[
    'site_country_code',
    'admin_project_code',
    'admin_data_partner',
    'admin_project_contact',
    'bldg_use_type',
    'bldg_use_subtype',
    'bldg_project_status',
    'site_country',
    'bldg_year_permit',
    'bldg_year_complete',
    'bldg_year_complete_interval',
    'bldg_QTO_type',
    'bldg_area_definition',
    'bldg_area_gfa','bldg_area_hfa',
    'bldg_area_interval',
    'bldg_users_total',
    'bldg_floors_ag',
    'bldg_floors_bg',
    'bldg_struct_type',
    'bldg_roof_type',
    'bldg_energy_class_general',
    'bldg_energy_class_country',
    'bldg_certification',
    
    'inv_energy_consumption',
    'inv_mat_mass_total',
    'inv_mat_mass_total_m2',
    'inv_mat_mass_total_capita',
    'inv_mat_mass_sum_top_5',
    'inv_mat_mass_ratio',
    'inv_mat_1_type',
    'inv_mat_1_mass',
    'inv_mat_2_type',
    'inv_mat_2_mass',
    'inv_mat_3_type',
    'inv_mat_3_mass',
    'inv_mat_4_type',
    'inv_mat_4_mass',
    'inv_mat_5_type',
    'inv_mat_5_mass',
    
    'mass_aluminium',
    'mass_bamboo',
    'mass_brass_copper',
    'mass_cement_mortar',
    'mass_ceramics',
    'mass_concrete_reinforced',
    'mass_concrete_wo_reinforcement',
    'mass_earth',
    'mass_EPS_XPS',
    'mass_fungi',
    'mass_glass',
    'mass_metals',
    'mass_plastics',
    'mass_steel_reinforcement',
    'mass_stone',
    'mass_stone_wool',
    'mass_straw_hemp',
    'mass_wood',
    'mass_other',
    
    'eurostat_metal_materials',
    'eurostat_non-metallic_minerals',
    'eurostat_fossil_energy_materials',
    'eurostat_biomass_based_materials',
    
    'lca_RSP',
    'lca_software',
    'lca_database',
    'lca_scenarios_decarbonisation',
    
    'scope_parts',
    'scope_LCS',
    'scope_LCM',    
    
    'scope_parts_1_ground',
    'scope_parts_2_structure',
    'scope_parts_3_secondary',
    'scope_parts_4_finishes',
    'scope_parts_5_mechanical',
    'scope_parts_6_electrical',
    'scope_parts_6+_renewables',
    'scope_parts_7_facilities',
    'scope_parts_8_fittings',
    
    'scope_LCS_A123',
    'scope_LCS_A4',
    'scope_LCS_A5',
    'scope_LCS_B1',
    'scope_LCS_B2',
    'scope_LCS_B3',
    'scope_LCS_B4',
    'scope_LCS_B5',
    'scope_LCS_B6',
    'scope_LCS_B7',
    'scope_LCS_B8',
    'scope_LCS_C1',
    'scope_LCS_C2',
    'scope_LCS_C3',
    'scope_LCS_C4',
    'scope_LCS_D',
    'scope_handling_D',
    
    'GHG_sum_em',
    'GHG_sum_em_m2a',
    'GHG_sum_em_capita_a',
    
    'GHG_A_m2a',
    'GHG_B_em_m2a',
    'GHG_B_op_m2a',
    'GHG_C_m2a',
        
    'GHG_A_capita_a',
    'GHG_B_em_capita_a',
    'GHG_B_op_capita_a',
    'GHG_C_capita_a',
    
#    'GHG_A1',
#    'GHG_A2',
#    'GHG_A3',
#    'GHG_A4',
#    'GHG_A5',
#    'GHG_B1',
#    'GHG_B2',
#    'GHG_B3',
#    'GHG_B4',
#    'GHG_B5',
#    'GHG_B6',
#    'GHG_B7',
#    'GHG_C1',
#    'GHG_C2',
#    'GHG_C3',
#    'GHG_C4',
#    'GHG_D',
    
    'GHG_A1_m2a',
    'GHG_A2_m2a',
    'GHG_A3_m2a',
    'GHG_A4_m2a',
    'GHG_A5_m2a',
    'GHG_B1_m2a',
    'GHG_B2_m2a',
    'GHG_B3_m2a',
    'GHG_B4_m2a',
    'GHG_B6_m2a',
    'GHG_B7_m2a',
    'GHG_C1_m2a',
    'GHG_C2_m2a',
    'GHG_C3_m2a',
    'GHG_C4_m2a',
    
#    'GHG_A123',
#    'GHG_A45',
#    'GHG_B1234',
#    'GHG_B67',
#    'GHG_C12',
#    'GHG_C34',
#    'GHG_C34_D',
    
    'GHG_A123_m2a', #Normal results per m2a
    'GHG_A45_m2a',
    'GHG_B1234_m2a',
    'GHG_B5_m2a',
    'GHG_B67_m2a',
    'GHG_C12_m2a',
    'GHG_C34_m2a',
    'GHG_D_m2a',
#    'GHG_C34_D_m2a',
    
    'GHG_A123_m2a_APEN', #APEN harmonized results per m2a
    'GHG_A45_m2a_APEN',
    'GHG_B1234_m2a_APEN',
    'GHG_B5_m2a_APEN',
    'GHG_B67_m2a_APEN',
    'GHG_C12_m2a_APEN',
    'GHG_C34_m2a_APEN',
    'GHG_D_m2a_APEN',
    'GHG_sum_em_m2a_APEN',
    'GHG_sum_op_m2a_APEN',
    
    'GHG_A123_m2a_harm', #Harmonized results per m2a
    'GHG_A45_m2a_harm',
    'GHG_B1234_m2a_harm',
    'GHG_B5_m2a_harm',
    'GHG_B67_m2a_harm',    
    'GHG_C12_m2a_harm',
    'GHG_C34_m2a_harm',
    'GHG_sum_em_m2a_harm',
    
#    'GHG_A123_m2_harm', #Harmonized results per m2 (no a) (disabled for now due to simplicity of recreating it on the fly)
#    'GHG_A45_m2_harm',
#    'GHG_B1234_m2_harm',
#    'GHG_B5_m2_harm',
#    'GHG_C12_m2_harm',
#    'GHG_C34_m2_harm',
#    'GHG_sum_em_m2_harm',
    
    'GHG_A123_m2a_harm_LCM', #Harmonized results per m2a with LCM ratio disaggregation applied where needed
    'GHG_A45_m2a_harm_LCM',
    'GHG_B1234_m2a_harm_LCM',
    'GHG_B5_m2a_harm_LCM',
    'GHG_C12_m2a_harm_LCM',
    'GHG_C34_m2a_harm_LCM',
    
    'GHG_P1_sum_m2a',
    'GHG_P1_A123_m2a',
    'GHG_P1_A45_m2a',
    'GHG_P1_B1234_m2a',
    'GHG_P1_B5_m2a',
    'GHG_P1_C12_m2a',
    'GHG_P1_C34_m2a',
    'GHG_P1_D_m2a',
    
    'GHG_P2_sum_m2a',
    'GHG_P2_A123_m2a',
    'GHG_P2_A45_m2a',
    'GHG_P2_B1234_m2a',
    'GHG_P2_B5_m2a',
    'GHG_P2_C12_m2a',
    'GHG_P2_C34_m2a',
    'GHG_P2_D_m2a',
    
    'GHG_P34_sum_m2a',
    'GHG_P34_A123_m2a',
    'GHG_P34_A45_m2a',
    'GHG_P34_B1234_m2a',
    'GHG_P34_B5_m2a',
    'GHG_P34_C12_m2a',
    'GHG_P34_C34_m2a',
    'GHG_P34_D_m2a',
    
    'GHG_P4_sum_m2a',
    'GHG_P4_A123_m2a',
    'GHG_P4_A45_m2a',
    'GHG_P4_B1234_m2a',
    'GHG_P4_B5_m2a',
    'GHG_P4_C12_m2a',
    'GHG_P4_C34_m2a',
    'GHG_P4_D_m2a',
    
    'GHG_P56_sum_m2a',
    'GHG_P56_A123_m2a',
    'GHG_P56_A45_m2a',
    'GHG_P56_B1234_m2a',
    'GHG_P56_B5_m2a',
    'GHG_P56_C12_m2a',
    'GHG_P56_C34_m2a',
    'GHG_P56_D_m2a',
    
    'GHG_P78_sum_m2a',
    'GHG_P78_A123_m2a',
    'GHG_P78_A45_m2a',
    'GHG_P78_B1234_m2a',
    'GHG_P78_B5_m2a',
    'GHG_P78_C12_m2a',
    'GHG_P78_C34_m2a',
    'GHG_P78_D_m2a',
    
    'GHG_sum_em_m2pertonGHG_a',
    'GHG_sum_em_m2pertonGHG',
    'GHG_sum_op_m2pertonGHG_a',
    'GHG_sum_op_m2pertonGHG',
    'GHG_sum_em_cappertonGHG_a',
    'GHG_sum_em_cappertonGHG',
    'GHG_sum_op_cappertonGHG_a',
    'GHG_sum_op_cappertonGHG',

    'ratio_A123_LCM',
    'ratio_A45_LCM',
    'ratio_B1234_LCM',
    'ratio_B5_LCM',
    'ratio_C12_LCM',
    'ratio_C34_LCM',
    
    'ratio_A123_parts',
    'ratio_A45_parts',
    'ratio_B1234_parts',
    'ratio_B5_parts',
    'ratio_C12_parts',
    'ratio_C34_parts' 
    
]]

#

In [53]:
#print(df.shape)
#print(df.dtypes)

# Create CSV

In [54]:
#Create csv
filename = '00_data/3_data_feature_engineered/EU-ECB_dataset_feature_engineered.csv'
df.to_csv(filename, index=False)

In [ ]:
print("Data has been feature engineered and ready for analysis.")